# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
#pip install gmap

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import json
import os
from IPython.display import display
import ipywidgets as widgets

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
#cities_df = pd.read_csv('../output_data/weather_data.csv')
cities_df = pd.read_csv('output_data/weather_data.csv', index_col = 0)
cities_df.head()

,City,Country,Lat,Lng,Max Temp (F),Cloudiness,Date,Humidity,Wind Speed
0,Hermanus,ZA,-34.4187,19.2345,59.81,22,1666373509,73,10.65
1,Luena,AO,-11.7833,19.9167,74.98,89,1666373529,40,3.33
2,Rikitea,PF,-23.1203,-134.9692,71.26,3,1666373530,72,22.95
3,Savonlinna,FI,61.8699,28.8800,30.25,20,1666373530,93,2.30
4,Hobart,AU,-42.8794,147.3294,62.46,100,1666373402,94,5.75


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
figure_layout = {
    'width': '400px',
    'height': '800px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

In [5]:
locations = cities_df[['Lat', 'Lng']]
weights = cities_df['Humidity']

# Plot Heatmap/Create heat layer
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights,dissipating=False, max_intensity=100, point_radius=3))

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
narrow_city = cities_df.loc[(cities_df["Max Temp (F)"] < 100) & (cities_df["Max Temp (F)"] > 0)].dropna()


narrow_city = narrow_city.iloc[0:15]

narrow_city


,City,Country,Lat,Lng,Max Temp (F),Cloudiness,Date,Humidity,Wind Speed
0,Hermanus,ZA,-34.4187,19.2345,59.81,22,1666373509,73,10.65
1,Luena,AO,-11.7833,19.9167,74.98,89,1666373529,40,3.33
2,Rikitea,PF,-23.1203,-134.9692,71.26,3,1666373530,72,22.95
3,Savonlinna,FI,61.8699,28.8800,30.25,20,1666373530,93,2.30
4,Hobart,AU,-42.8794,147.3294,62.46,100,1666373402,94,5.75
5,Naugachhia,IN,25.4000,87.1000,73.56,0,1666373530,65,3.33
6,Avarua,CK,-21.2078,-159.7750,77.05,75,1666373531,88,16.11
7,Puerto Baquerizo Moreno,EC,-0.9000,-89.6000,74.39,40,1666373531,73,16.11
8,Bluff,NZ,-46.6000,168.3333,41.61,9,1666373531,83,8.46
9,Okhotsk,RU,59.3833,143.3000,24.21,96,1666373532,62,14.58


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
hotel_df = narrow_city.copy()
hotel_df["Hotel Name"] = np.nan
hotel_df

,City,Country,Lat,Lng,Max Temp (F),Cloudiness,Date,Humidity,Wind Speed,Hotel Name
0,Hermanus,ZA,-34.4187,19.2345,59.81,22,1666373509,73,10.65,NaN
1,Luena,AO,-11.7833,19.9167,74.98,89,1666373529,40,3.33,NaN
2,Rikitea,PF,-23.1203,-134.9692,71.26,3,1666373530,72,22.95,NaN
3,Savonlinna,FI,61.8699,28.8800,30.25,20,1666373530,93,2.30,NaN
4,Hobart,AU,-42.8794,147.3294,62.46,100,1666373402,94,5.75,NaN
5,Naugachhia,IN,25.4000,87.1000,73.56,0,1666373530,65,3.33,NaN
6,Avarua,CK,-21.2078,-159.7750,77.05,75,1666373531,88,16.11,NaN
7,Puerto Baquerizo Moreno,EC,-0.9000,-89.6000,74.39,40,1666373531,73,16.11,NaN
8,Bluff,NZ,-46.6000,168.3333,41.61,9,1666373531,83,8.46,NaN
9,Okhotsk,RU,59.3833,143.3000,24.21,96,1666373532,62,14.58,NaN


In [8]:
place_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
param = {"key": g_key, "radius": 5000, "type": "Lodging"}
for i, row in hotel_df.iterrows():
    #print (row[['Lat', 'Lng']])
    param["location"] = f'{row["Lat"]},{row["Lng"]}'
    #print (param)
    try:
        response = requests.get(place_url, param).json()
        name = response['results'][0]['name']
        hotel_df.loc[i, "Hotel Name"] = name
    except:
        print("--Can't find hotel--")
            

In [9]:
hotel_df

,City,Country,Lat,Lng,Max Temp (F),Cloudiness,Date,Humidity,Wind Speed,Hotel Name
0,Hermanus,ZA,-34.4187,19.2345,59.81,22,1666373509,73,10.65,Hermanus
1,Luena,AO,-11.7833,19.9167,74.98,89,1666373529,40,3.33,Luena
2,Rikitea,PF,-23.1203,-134.9692,71.26,3,1666373530,72,22.95,Rikitea
3,Savonlinna,FI,61.8699,28.8800,30.25,20,1666373530,93,2.30,Savonlinna
4,Hobart,AU,-42.8794,147.3294,62.46,100,1666373402,94,5.75,Hobart
5,Naugachhia,IN,25.4000,87.1000,73.56,0,1666373530,65,3.33,Naugachhia
6,Avarua,CK,-21.2078,-159.7750,77.05,75,1666373531,88,16.11,Avarua District
7,Puerto Baquerizo Moreno,EC,-0.9000,-89.6000,74.39,40,1666373531,73,16.11,Puerto Baquerizo Moreno
8,Bluff,NZ,-46.6000,168.3333,41.61,9,1666373531,83,8.46,Bluff
9,Okhotsk,RU,59.3833,143.3000,24.21,96,1666373532,62,14.58,Okhotsk


In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
hotel_info

['\n<dl>\n<dt>Name</dt><dd>Hermanus</dd>\n<dt>City</dt><dd>Hermanus</dd>\n<dt>Country</dt><dd>ZA</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Luena</dd>\n<dt>City</dt><dd>Luena</dd>\n<dt>Country</dt><dd>AO</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Rikitea</dd>\n<dt>City</dt><dd>Rikitea</dd>\n<dt>Country</dt><dd>PF</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Savonlinna</dd>\n<dt>City</dt><dd>Savonlinna</dd>\n<dt>Country</dt><dd>FI</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Hobart</dd>\n<dt>City</dt><dd>Hobart</dd>\n<dt>Country</dt><dd>AU</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Naugachhia</dd>\n<dt>City</dt><dd>Naugachhia</dd>\n<dt>Country</dt><dd>IN</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Avarua District</dd>\n<dt>City</dt><dd>Avarua</dd>\n<dt>Country</dt><dd>CK</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Puerto Baquerizo Moreno</dd>\n<dt>City</dt><dd>Puerto Baquerizo Moreno</dd>\n<dt>Country</dt><dd>EC</dd>\n</dl>\n',
 '\n<dl>\n<dt>Name</dt><dd>Bluff</dd>\n<dt>City</dt><dd>Bluff</dd

In [12]:
# Add marker layer ontop of heat map
#fig = gmaps.figure()
hotel_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(hotel_layer)
fig
# Display figure


Figure(layout=FigureLayout(height='420px'))